In [1]:
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.cross_validation import train_test_split
from sklearn.datasets import load_svmlight_file
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.externals import joblib
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.externals import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from scipy.sparse import hstack
import time, os, random, sys
import math
import hyperopt.tpe
import hpsklearn.components
import hpsklearn.demo_support
random.seed(1)

/Users/garb47/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/garb47/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


WARN: OMP_NUM_THREADS=None =>
... If you are using openblas if you are using openblas set OMP_NUM_THREADS=1 or risk subprocess calls hanging indefinitely


In [9]:
def gbdt_lr_train_test(libsvmFileName):
    train_df, test_df = train_test_split(example, train_size = 0.8)
    X_train = train_df.drop(['Unnamed: 0', 'Grant.Status'], axis=1)
    y_train = train_df['Grant.Status']
    X_test = test_df.drop(['Unnamed: 0', 'Grant.Status'], axis=1)
    y_test = test_df['Grant.Status']
    gbclf = GradientBoostingClassifier(n_estimators=30, max_depth=4, verbose=0)
    tuned_parameter = [{'n_estimators':[30, 40, 50, 60], 'max_depth':[3, 4, 5, 6], 'max_features':[0.5,0.7,0.9]}]
    gs_clf = GridSearchCV(gbclf, tuned_parameter, cv=5, scoring='roc_auc')
    gs_clf.fit(X_train, y_train)
    print('best parameters set found: ')
    print(gs_clf.best_params_)
    y_pred_gbdt = gs_clf.predict_proba(X_test)[:, 1]
    gbdt_auc = roc_auc_score(y_test, y_pred_gbdt)
    print('gbdt auc: %.5f' % gbdt_auc)
    X_train_leaves = gbclf.fit(gbclf.apply(X_train)[:,:,0])
    (train_rows, cols) = X_train_leaves.shape
    X_test_leaves = gbclf.fit(gbclf.apply(X_test)[:,:,0])
    gbdtenc = OneHotEncoder()
    X_trans = gbdtenc.fit_transform(np.concatenate((X_train_leaves, X_test_leaves), axis=0))
    lr = LogisticRegression()
    lr.fit(X_trans[:train_rows, :], y_train)
    y_pred_gbdtlr1 = lr.predict_proba(X_trans[train_rows:, :])[:, 1]
    gbdtlr_auc1 = roc_auc_score(y_test, y_pred_gbdtlr1)
    print('gbdt+lr auc 1: %.5f' % gbdtlr_auc1)
    lr = LogisticRegression(n_jobs=-1)
    X_train_ext = hstack([X_trans[:train_rows, :], X_train])
    lr.fit(X_train_ext, y_train)
    X_test_ext = hstack([X_trans[train_rows:, :], X_test])
    y_pred_gbdtlr2 = lr.predict_proba(X_test_ext)[:, 1]
    gbdtlr_auc2 = roc_auc_score(y_test, y_pred_gbdtlr2)
    print('gbdt+lr auc 2: %.5f' % gbdtlr_auc2)

In [3]:
example = pd.read_csv('example.csv')

In [10]:
gbdt_lr_train_test(example)

best parameters set found: 
{'max_depth': 6, 'max_features': 0.9, 'n_estimators': 60}
gbdt auc: 0.93128


NotFittedError: This GradientBoostingClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.